# Paradise Papers

NOTE: the original walkthrough is available using `:play https://guides.neo4j.com/sandbox/icij-offshoreleaks/paradise-papers-examples.html` in Neo4j Browser. I updated some of the Cypher queries that cannot run in Neo4j 5.

The demo uses Docker to run. You can execute [run.sh](./run.sh) to start the container.

## Overview

Meta Graph:
```sql
CALL db.schema.visualization()
```
Counts per entity type:
```sql
MATCH (node)
RETURN labels(node) AS type,count(*)
```

Which intermediaries have the most connections to which entities?
```sql
MATCH (i:Intermediary) WHERE COUNT{(i)--()} > 100 AND i.sourceID STARTS WITH "Paradise Papers"
MATCH (i)-[connection]-(entity)
RETURN i.name as intermediary, type(connection) as relationship, head(labels(entity)) as type, count(*) as count
ORDER BY count DESC LIMIT 20;
```

![](img/003.png)

## How Graphs Helped our Investigation

For example, let's say we want to discover the shortest paths between two entity officers through a set of Entity or Address nodes. This is quite easy with Cypher, Neo4j's graph query language.

```sql
MATCH (a:Officer),(b:Officer)
          WHERE a.name CONTAINS 'Ross, Jr' 
            AND b.name CONTAINS 'Grant' AND a.sourceID STARTS WITH "Paradise Papers"
          MATCH p=allShortestPaths((a)-[:officer_of|intermediary_of|registered_address*..10]-(b))
          RETURN p LIMIT 50;
```

![](img/001.png)

Jurisdiction distribution of intermediaries in the ICIJ offshore leaks DB (Appleby is an offshore law firm):

```sql
MATCH (intermediary:Intermediary)-[:intermediary_of]->(e:Entity)
          WHERE intermediary.name CONTAINS 'Appleby' AND intermediary.sourceID STARTS WITH "Paradise Papers"
          RETURN e.jurisdiction AS country, COUNT(*) AS frequency
          ORDER BY frequency DESC LIMIT 20;
```
Result:

![](img/002.png)

Entities registered by an Intermediary (Appleby):

```sql
MATCH (i:Intermediary)-[r:intermediary_of]->(e:Entity)
WHERE i.name CONTAINS "Appleby"
RETURN i, r, e LIMIT 100;
```

Result:

![](img/004.png)

Officers with most entities:

```sql
MATCH (o:Officer)-[:officer_of]->(:Entity)
WHERE o.sourceID STARTS WITH "Paradise Papers"
RETURN o.name, count(*) as entities
ORDER BY entities DESC LIMIT 10;
```

Result:

![](img/005.png)

### Search for Officer nodes by name

Enter any name (e.g. from ICIJ [published investigations](https://www.icij.org/investigations/)) into the form then click on the query to execute to see if that person appears in the data. Note that this search is case sensitive and searches exact matches only. We're setting a parameter for the officer which we can reuse later, just click and run the `:param` block.

```sql
:param officer=>"Smith"

MATCH (o:Officer)
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Paradise Papers"
RETURN o
LIMIT 100;
```

Let’s see with which entities our officer was involved with, including first and second degree connections.

**First degree:**

```sql
MATCH (o:Officer)
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Paradise Papers"
MATCH path = (o)-[r]->(:Entity)
RETURN path LIMIT 100
```

Result:

![](img/006.png)

**Second degree entities:**

```sql
MATCH (o:Officer) 
WHERE o.name CONTAINS $officer AND o.sourceID STARTS WITH "Paradise Papers"
MATCH path = (o)-[]->(:Entity)<-[]-(:Officer)-[]->(:Entity)
RETURN path LIMIT 100;
```

Result:

![](img/007.png)

### Find who is behind an Entity and the roles that they play

Name of entity: Appleby.

```sql
MATCH (e:Entity)-[r]-(o:Officer)
WHERE e.name CONTAINS $entity AND o.sourceID STARTS WITH "Paradise Papers"
RETURN *
LIMIT 100
```

### Joint involvement

When investigating, it is very important to identify people that appear to operate together. You can try to find if two officers appear connected to the same entities recurrently by using this query. Note that in this case, the first results show companies because companies can also be officers of entities.

```sql
MATCH (o1:Officer)-[r1]->(e:Entity)<-[r2]-(o2:Officer)
USING JOIN ON e
WHERE id(o1) < id(o2) AND COUNT{ (o1)-->() } > 10 AND COUNT{ (o2)-->() } > 10 AND o1.sourceID STARTS WITH "Paradise Papers"
WITH o1,o2,count(*) as freq, collect(e)[0..10] as entities
WHERE freq > 10
WITH * ORDER BY freq DESC LIMIT 10
RETURN o1.name, o2.name, freq, [e IN entities | e.name];
```

Result:

![](img/008.png)

We can attempt to remove companies from the results by filtering our Officer names that contain strings commonly found in company names such as "LLC", "Limited", "Ltd", etc:

```sql
MATCH (o1:Officer)-[r1]->(e:Entity)<-[r2]-(o2:Officer)
USING JOIN ON e
WHERE id(o1) < id(o2) AND NOT o1.name CONTAINS "LIMITED" AND NOT o1.name CONTAINS "Limited" AND o1.sourceID STARTS WITH "Paradise Papers"
AND NOT o2.name CONTAINS "Limited" AND NOT o2.name CONTAINS "LIMITED"
AND COUNT{ (o1)-->() } > 10 AND COUNT{ (o2)-->() } > 10
WITH o1,o2,count(*) as freq, collect(e)[0..10] as entities
WHERE freq > 10
WITH * ORDER BY freq DESC LIMIT 10
RETURN o1.name, o2.name, freq, [e IN entities | e.name]
```

![](img/009.png)

Visualize the graph:

```sql
MATCH (o1:Officer)-[r1]->(e:Entity)<-[r2]-(o2:Officer)
USING JOIN ON e
WHERE id(o1) < id(o2) AND NOT o1.name CONTAINS "LIMITED" AND NOT o1.name CONTAINS "Limited" AND o1.sourceID STARTS WITH "Paradise Papers"
AND NOT o2.name CONTAINS "Limited" AND NOT o2.name CONTAINS "LIMITED"
AND COUNT{ (o1)-->() } > 10 AND COUNT{ (o2)-->() } > 10
WITH o1,o2,count(*) as freq, collect(e)[0..10] as entities
WHERE freq > 200
RETURN o1,o2,apoc.create.vRelationship(o1,'JOINT',{freq:freq,entities:[e IN entities| e.name]},o2)
```

![](img/010.png)

### Shortest path between two people

Name of the first officer: Smith, Name of the second officer: Grant

```sql
MATCH (a:Officer),(b:Officer)
WHERE a.name CONTAINS 'Smith' AND b.name CONTAINS 'Grant' AND a.sourceID STARTS WITH "Paradise Papers"
WITH a,b LIMIT 20000
MATCH p=allShortestPaths((a)-[:officer_of|intermediary_of|registered_address*..10]-(b))
RETURN p
LIMIT 50;
```


### Query by address

We can use Neo4j's string comparison functions to search for addresses that contain cities and countries in which we are interested:

```sql
MATCH (a:Address)<-[:registered_address]-(other)
WHERE a.address CONTAINS 'Barcelona' AND a.countries CONTAINS 'Spain' AND a.sourceID STARTS WITH "Paradise Papers"
RETURN a, other
LIMIT 100
```

### Offshore entity juridictions by intermediary

What are the most popular offshore jurisdictions for certain intermediaries?

```sql
MATCH (a:Address)<-[:registered_address]-(other)
WHERE a.address CONTAINS 'Barcelona' AND a.countries CONTAINS 'Spain' AND a.sourceID STARTS WITH "Paradise Papers"
RETURN a, other
LIMIT 100
```

![Result](img/011.png)

### Most popular offshore jurisdiction for people connected to a country

What are the common offshore jurisdictions for officers with addresses in a certain country?

```sql
MATCH (o:Officer)-->(e:Entity)<-[:intermediary_of]-(i:Intermediary)
WHERE o.country_codes CONTAINS 'USA' AND o.sourceID STARTS WITH "Paradise Papers"
RETURN e.jurisdiction_description AS jurisdiction, count(*) AS number
ORDER BY number DESC LIMIT 10;
```

![](img/012.png)

## Full Text Search

All the previous queries we run were searching the exact text of a node. However, you can also build an index to search the text of properties inside a node. Just like in a book where you look at the index to find a section that interest you, and then start reading from there.

For that, you need to build the index first. It will take a little while since the procedure has to read through the entire database to create it.

We used Solr as a full text search engine to search across all data. But you can do the same in Neo4j directly, both use Apache Lucene under the hood for full text search.

In order to use the full text search feature, we indexed our data by specifying all the properties we want to index. Here we created a full text index called search (we will use this name when searching in the index) with all our data. You don't have to run this, it's just here as an example:

```sql
CREATE FULLTEXT INDEX search FOR (n:Officer|Intermediary|Address|Entity) ON EACH [n.name, n.address];
```

Then:

```sql
CALL db.index.fulltext.queryNodes('search','Nike',{limit:10})
```

![](img/013.png)

You can futher restrict the full text search to only searching in a particular property. In order to seach for an Entity incorporated by offshore service provider Appleby, use the following:

```sql
CALL db.index.fulltext.queryNodes('search', 'name:"Appleby"',{limit:10})
```

### Full text search with graph patterns

Previously we searched for nodes by matching against property values. However, integrating text search with an graph query is so much more powerful.

We could for instance search for addresses in the database that contain the word "New York", and then find all entities registered at those addresses:

```sql
CALL db.index.fulltext.queryNodes("search", 'address:"New York"')
            YIELD node AS addr WHERE addr.sourceID STARTS WITH "Paradise Papers"
            MATCH (addr:Address)<-[:registered_address]-(entity)
            RETURN addr, entity LIMIT 50;
```
![](img/014.png)

There may be typos in the data so we can use fuzzy matching to find addresses that have inconsistent spellings. Add a tilde (~) to instruct the index search procedure to do a fuzzy match, allowing you to find “Malta” even if the spelling is slightly off.

```sql
CALL db.index.fulltext.queryNodes("search", 'address:Malta~')
            YIELD node AS addr WHERE addr.sourceID STARTS WITH "Paradise Papers"
            MATCH (addr:Address)<-[:registered_address]-(entity)
            RETURN addr, entity LIMIT 50;
```

![](img/015.png)


You might notice that there are addresses that contain the word “Barcelona” that are not in Barcelona, Spain.

We can further specify that we want the text to contain both the word Barcelona, AND the word Spain with:

```sql
CALL db.index.fulltext.queryNodes("search", '+name:Barcelona +name:Spain')
            YIELD node AS addr WHERE addr.sourceID STARTS WITH "Paradise Papers"
            MATCH (addr:Address)<-[:registered_address]-(entity)
            RETURN addr, entity LIMIT 50
```

For more details on the query syntax used in the second parameter of the `search` procedure, please see this [Lucene query tutorial](https://lucenetutorial.com/lucene-query-syntax.html)

## Graph Analytics

PageRank is an algorithm used for instance by Google to rank websites in their search engine results. It assumes that more important entities are likely to have more connections pointing to them, and not just directly but also indirectly.

The algorithm can also be used as way of measuring the importance of entities in a network. Let’s use PageRank to find the top 20 ranked entities in the dataset (this runs for about 20s).

```sql
CALL gds.graph.project(
    'data-projection',
    '*',
    '*'
)
YIELD
  graphName AS graph,
  relationshipProjection AS knowsProjection,
  nodeCount AS nodes,
  relationshipCount AS rels;
```

```sql
CALL gds.pageRank.stream(
    'data-projection'
)
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
WHERE node:Entity AND node.sourceID STARTS WITH "Paradise Papers"
RETURN node.name AS entity, score
ORDER BY score DESC 
LIMIT 20;
```

![](img/016.png)

Now let’s try a similar query but investigating different patterns. The following detects the 10 top ranked addresses. It then finds how many entities are registered at those addresses. Note that the address nodes contain text that has not been standardized, therefore there may be inconsistencies in the address data.

```sql
CALL gds.pageRank.stream(
    'data-projection'
)
YIELD nodeId, score
WITH gds.util.asNode(nodeId) as node, score
        WHERE node:Address AND node.sourceID STARTS WITH "Paradise Papers"
WITH * ORDER BY score DESC  LIMIT 10
MATCH (address)<-[:registered_address]-(e:Entity)
RETURN address.address, count(e) AS count
ORDER BY count DESC
```

![](img/017.png)